## FIRMS API configuration

In [1]:
API_KEY = "b3b38f287a8ba9db57fac29012ed1e9c"
FIRMS_URL = f"https://firms.modaps.eosdis.nasa.gov/api/country/csv/{API_KEY}/VIIRS_NOAA21_NRT/USA/1"

In [2]:
import pandas as pd
from ipyleaflet import Map,GeoData
from apscheduler.schedulers.background import BackgroundScheduler
from IPython.display import display, clear_output
import geopandas
# Create a map centered around the USA
map = Map(center=(37.0902, -95.7129), zoom=4)

## Function to fetch and process data into a DataFrame

In [3]:
def fetch_and_process_fire_data():
    df=pd.read_csv(FIRMS_URL)
    df_subset= df[['latitude','longitude','bright_ti4','acq_date','acq_time','confidence','frp']]
    filtered_df=df_subset[df_subset['confidence']!='l']
    geodf=geopandas.GeoDataFrame(
    filtered_df, geometry=geopandas.points_from_xy(filtered_df.longitude, filtered_df.latitude))
    return geodf

## Function to schedule the task

In [ ]:
def scheduled_task():
    gdf=fetch_and_process_fire_data()
    # print(gdf.tail())
    # Clear previous output and display the updated map
    clear_output(wait=True)
    
    geo_data = GeoData(geo_dataframe = gdf)
    # Add new data to the leaflet map
    map.add(geo_data)
    
    # Display the updated map
    display(map)


## Set up Scheduler

In [5]:
scheduler = BackgroundScheduler()
scheduler.add_job(scheduled_task, 'interval', minutes=1)  # Fetch every minute
scheduler.start()
# Display the initial map
scheduled_task()

Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

     latitude  longitude  bright_ti4    acq_date  acq_time confidence    frp  \
578  46.56181 -112.14559      332.68  2025-01-15      1951          n  11.52   
579  46.56547 -112.14744      328.65  2025-01-15      1951          n  11.52   
580  46.56695 -112.14050      348.79  2025-01-15      1951          n  64.98   
581  46.57059 -112.14239      351.76  2025-01-15      1951          n  27.00   
582  47.35494 -101.83327      327.89  2025-01-15      1951          n   2.57   

                        geometry  
578  POINT (-112.14559 46.56181)  
579  POINT (-112.14744 46.56547)  
580   POINT (-112.1405 46.56695)  
581  POINT (-112.14239 46.57059)  
582  POINT (-101.83327 47.35494)  


In [ ]:
# Keep the Jupyter Notebook running so that APScheduler can execute tasks
import time
while True:
    time.sleep(1)

Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…